In [2]:
import pandas as pd
import statsmodels.formula.api as smf


In [4]:
df = pd.read_csv('../data/full_survey_data.csv')

/var/folders/b3/glj_22g92pl25nkdvjvxnzz00000gn/T/ipykernel_3942/4096187165.py:1: DtypeWarning: Columns (10,11,12,15,16,17,18,19,20,21,22,24,27,37,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,69,74,75,76,78,88,91,94,95,104,113,114,115,116,118,121,122,123,124,125,132,135,137,138,141,142,151,160,161,162,163,165,168,169,170,171,172,179,182,184,185,188,189,198,207,208,209,210,212,215,216,217,218,219,226,229,231,232,236,245,254,255,256,257,259,262,263,264,265,266,273,276,278,279,283,292,301,302,303,304,306,309,310,312,313,320,325,326,330,339,348,349,350,351,353,356,357,359,360,367,372,373,377,386,395,396,397,398,400,403,404,406,407,414,419,420,424,433,443,444,445,446,447,448,450,451,452,453,454,455,456,458,459,460,461,462,463,464,465,466,467,469,470,471,472,473,474,475,476,478,479,480,481,482,500,501,502,508,510,515,516,522,524,525,526,533,538,547,556,561,565,566,567,574,580,581,582,591,600,603,607,608,609,616,620,624,633,680,681,682,687,688,694,696,697,698,705,710

In [5]:


# Create a list for all crop_name and crop_residue_use columns
crop_cols = [f"crop_name_{i}" for i in range(1, 9)]
residue_cols = [f"crop_residue_use_{i}" for i in range(1, 9)]

# Create a list of tuples for each crop's pair of name and residue
records = []
for i in range(1, 9):
    temp_df = df[['id_unique', f'crop_name_{i}', f'crop_residue_use_{i}',f'crop_harvest_kg_per_year_{i}']].copy()
    temp_df.columns = ['id_unique', 'crop_name', 'crop_residue_use','crop_harvest_kg_per_year']
    temp_df['crop_number'] = i
    records.append(temp_df)

# Concatenate all together
long_df = pd.concat(records, ignore_index=True)

# Drop rows where crop_name is NaN
#long_df = long_df.dropna(subset=['crop_name'])

#sort
long_df = long_df.sort_values(['id_unique', 'crop_number']).reset_index(drop=True)
long_df

,id_unique,crop_name,crop_residue_use,crop_harvest_kg_per_year,crop_number
0,bf_adn_2019_10_1,millet,soil feed,1200.0,1
1,bf_adn_2019_10_1,groundnut,soil feed,NaN,2
2,bf_adn_2019_10_1,sesame,soil feed,100.0,3
3,bf_adn_2019_10_1,NaN,NaN,NaN,4
4,bf_adn_2019_10_1,NaN,NaN,NaN,5
...,...,...,...,...,...
438979,zm_scn_2017_9_1,NaN,NaN,NaN,4
438980,zm_scn_2017_9_1,NaN,NaN,NaN,5
438981,zm_scn_2017_9_1,NaN,NaN,NaN,6
438982,zm_scn_2017_9_1,NaN,NaN,NaN,7


In [6]:
final_crop_df = long_df[(long_df.crop_name.notna()) & (long_df.crop_residue_use.notna())]
final_crop_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95962 entries, 0 to 438976
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id_unique                 95962 non-null  object 
 1   crop_name                 95962 non-null  object 
 2   crop_residue_use          95962 non-null  object 
 3   crop_harvest_kg_per_year  92536 non-null  float64
 4   crop_number               95962 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 4.4+ MB


In [7]:
final_crop_df['burn_crop'] = final_crop_df['crop_residue_use'].str.contains('burn', case=False, na=False).astype(int)

/var/folders/b3/glj_22g92pl25nkdvjvxnzz00000gn/T/ipykernel_3942/2510554493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_crop_df['burn_crop'] = final_crop_df['crop_residue_use'].str.contains('burn', case=False, na=False).astype(int)


In [10]:
final_crop_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95962 entries, 0 to 438976
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id_unique                 95962 non-null  object 
 1   crop_name                 95962 non-null  object 
 2   crop_residue_use          95962 non-null  object 
 3   crop_harvest_kg_per_year  92536 non-null  float64
 4   crop_number               95962 non-null  int64  
 5   burn_crop                 95962 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 5.1+ MB


In [9]:
# Step 1: Select relevant columns
name_cols = [f'livestock_name_{i}' for i in range(1, 6)]
died_cols = [f'livestock_died_{i}' for i in range(1, 6)]

# Step 2: Melt the livestock names and deaths
names_long = df.melt(id_vars='id_unique', value_vars=name_cols, 
                     var_name='animal_slot', value_name='livestock_name')

died_long = df.melt(id_vars='id_unique', value_vars=died_cols, 
                    var_name='animal_slot', value_name='livestock_died')

# Step 3: Make sure they align by extracting index number
names_long['slot'] = names_long['animal_slot'].str.extract(r'(\d+)').astype(int)
died_long['slot'] = died_long['animal_slot'].str.extract(r'(\d+)').astype(int)

# Step 4: Merge the two long DataFrames
long_df = pd.merge(
    names_long[['id_unique', 'slot', 'livestock_name']],
    died_long[['id_unique', 'slot', 'livestock_died']],
    on=['id_unique', 'slot']
)

# Optional: drop rows where animal name is missing
long_df = long_df.dropna(subset=['livestock_name']).reset_index(drop=True)

# Result: Each row is (id_unique, livestock_name, livestock_died)
print(long_df.head())

death_df = long_df[long_df.livestock_died.notna()]

         id_unique  slot livestock_name  livestock_died
0  bf_adn_2019_1_1     1          sheep             0.0
1  bf_adn_2019_2_1     1         cattle             0.0
2  bf_adn_2019_3_1     1           pigs             3.0
3  bf_adn_2019_4_1     1         cattle             0.0
4  bf_adn_2019_5_1     1         cattle             0.0


In [11]:
# List of animals to keep
animals_to_keep = ['cattle', 'chicken', 'goats', 'sheep', 'donkeys_horses', 'buffalo', 'pigs']

# Filter the DataFrame
final_death_df = death_df[death_df['livestock_name'].isin(animals_to_keep)]
final_death_df

,id_unique,slot,livestock_name,livestock_died
0,bf_adn_2019_1_1,1,sheep,0.0
1,bf_adn_2019_2_1,1,cattle,0.0
2,bf_adn_2019_3_1,1,pigs,3.0
3,bf_adn_2019_4_1,1,cattle,0.0
4,bf_adn_2019_5_1,1,cattle,0.0
...,...,...,...,...
81610,tz_glv_2017_156_1,5,donkeys_horses,0.0
81614,tz_glv_2017_522_1,5,pigs,0.0
81615,tz_glv_2017_562_1,5,donkeys_horses,0.0
81616,tz_glv_2017_889_1,5,chicken,3.0


In [12]:
# Sum livestock deaths per farm
deaths_by_farm = (
    final_death_df.groupby("id_unique")["livestock_died"]
    .sum()
    .reset_index(name="total_livestock_died")
)

deaths_by_farm.head()

,id_unique,total_livestock_died
0,bf_adn_2019_10_1,3.0
1,bf_adn_2019_11_1,5.0
2,bf_adn_2019_12_1,1.0
3,bf_adn_2019_13_1,3.0
4,bf_adn_2019_14_1,7.0


In [13]:
from scipy.stats.mstats import winsorize

# Extract the data column
values = deaths_by_farm["total_livestock_died"].values

# Winsorize only the top 5%
winsorized_values = winsorize(values, limits=(0, 0.05))

In [14]:
# Create a new column with the winsorized values
deaths_by_farm["winsorized_deaths"] = winsorized_values

In [15]:
# Group by farm and take max of burn_crop (since 1 means burning occurred)
burning_by_farm = (
    final_crop_df.groupby("id_unique")["burn_crop"]
    .max()
    .reset_index(name="any_burning")
)

burning_by_farm.head()

,id_unique,any_burning
0,bf_adn_2019_10_1,0
1,bf_adn_2019_11_1,0
2,bf_adn_2019_12_1,0
3,bf_adn_2019_13_1,0
4,bf_adn_2019_14_1,0


In [16]:
merged_farm_df = pd.merge(burning_by_farm, deaths_by_farm, on="id_unique", how="inner")

In [17]:
merged_farm_df

,id_unique,any_burning,total_livestock_died,winsorized_deaths
0,bf_adn_2019_10_1,0,3.0,3.0
1,bf_adn_2019_11_1,0,5.0,5.0
2,bf_adn_2019_12_1,0,1.0,1.0
3,bf_adn_2019_13_1,0,3.0,3.0
4,bf_adn_2019_14_1,0,7.0,7.0
...,...,...,...,...
31360,zm_scn_2017_95_1,0,0.0,0.0
31361,zm_scn_2017_96_1,0,1.0,1.0
31362,zm_scn_2017_98_1,0,0.0,0.0
31363,zm_scn_2017_99_1,0,1.0,1.0


In [18]:
merged_farm_df.describe()

,any_burning,total_livestock_died,winsorized_deaths
count,31365.000000,3.136500e+04,31365.000000
mean,0.146246,1.392799e+02,2.946628
std,0.353358,2.259231e+04,5.293842
min,0.000000,0.000000e+00,0.000000
25%,0.000000,0.000000e+00,0.000000
50%,0.000000,0.000000e+00,0.000000
75%,0.000000,3.000000e+00,3.000000
max,1.000000,4.000000e+06,20.000000


In [19]:
# Crop count
merged_farm_df = merged_farm_df.merge(
    df[["id_unique", "crop_count"]],
    on="id_unique",
    how="left"
)

# Harvest early
merged_farm_df = merged_farm_df.merge(
    df[["id_unique", "harvest_early"]],
    on="id_unique",
    how="left"
)

# Food shortage time
merged_farm_df = merged_farm_df.merge(
    df[["id_unique", "foodshortagetime"]],
    on="id_unique",
    how="left"
)

# Debts have
merged_farm_df = merged_farm_df.merge(
    df[["id_unique", "debts_have"]],
    on="id_unique",
    how="left"
)

# Count people
merged_farm_df = merged_farm_df.merge(
    df[["id_unique", "count_people"]],
    on="id_unique",
    how="left"
)

In [20]:
for col in ["harvest_early", "foodshortagetime", "debts_have"]:
    merged_farm_df[col] = merged_farm_df[col].map({"y": 1, "n": 0})
merged_farm_df

,id_unique,any_burning,total_livestock_died,winsorized_deaths,crop_count,harvest_early,foodshortagetime,debts_have,count_people
0,bf_adn_2019_10_1,0,3.0,3.0,3.0,1.0,1.0,0.0,9.0
1,bf_adn_2019_11_1,0,5.0,5.0,3.0,0.0,1.0,0.0,8.0
2,bf_adn_2019_12_1,0,1.0,1.0,3.0,0.0,1.0,0.0,6.0
3,bf_adn_2019_13_1,0,3.0,3.0,3.0,1.0,1.0,0.0,6.0
4,bf_adn_2019_14_1,0,7.0,7.0,3.0,0.0,1.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...
31360,zm_scn_2017_95_1,0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
31361,zm_scn_2017_96_1,0,1.0,1.0,3.0,0.0,1.0,1.0,NaN
31362,zm_scn_2017_98_1,0,0.0,0.0,1.0,0.0,1.0,1.0,NaN
31363,zm_scn_2017_99_1,0,1.0,1.0,1.0,0.0,1.0,1.0,NaN


In [21]:
merged_farm_df = pd.merge(
    merged_farm_df,
    df[["id_unique", "country", "year"]],
    on="id_unique",
    how="left"
)

merged_farm_df

,id_unique,any_burning,total_livestock_died,winsorized_deaths,crop_count,harvest_early,foodshortagetime,debts_have,count_people,country,year
0,bf_adn_2019_10_1,0,3.0,3.0,3.0,1.0,1.0,0.0,9.0,burkina_faso,2019
1,bf_adn_2019_11_1,0,5.0,5.0,3.0,0.0,1.0,0.0,8.0,burkina_faso,2019
2,bf_adn_2019_12_1,0,1.0,1.0,3.0,0.0,1.0,0.0,6.0,burkina_faso,2019
3,bf_adn_2019_13_1,0,3.0,3.0,3.0,1.0,1.0,0.0,6.0,burkina_faso,2019
4,bf_adn_2019_14_1,0,7.0,7.0,3.0,0.0,1.0,0.0,12.0,burkina_faso,2019
...,...,...,...,...,...,...,...,...,...,...,...
31360,zm_scn_2017_95_1,0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,zambia,2017
31361,zm_scn_2017_96_1,0,1.0,1.0,3.0,0.0,1.0,1.0,NaN,zambia,2017
31362,zm_scn_2017_98_1,0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,zambia,2017
31363,zm_scn_2017_99_1,0,1.0,1.0,1.0,0.0,1.0,1.0,NaN,zambia,2017


In [22]:
print("\n--- Base Model ---")
model = smf.ols(
    formula="any_burning ~ winsorized_deaths + C(year) + C(country)",
    data=merged_farm_df
).fit()
print(model.summary())


--- Base Model ---
                            OLS Regression Results                            
Dep. Variable:            any_burning   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     184.1
Date:                Fri, 15 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:32:31   Log-Likelihood:                -8715.9
No. Observations:               31365   AIC:                         1.751e+04
Df Residuals:                   31326   BIC:                         1.784e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [24]:
# crop_count
print("\n--- Add crop_count ---")
model = smf.ols(
    formula="any_burning ~ winsorized_deaths + C(year) + C(country) + crop_count",
    data=merged_farm_df
).fit()
print(model.summary())


--- Add crop_count ---
                            OLS Regression Results                            
Dep. Variable:            any_burning   R-squared:                       0.191
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     189.2
Date:                Fri, 15 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:34:49   Log-Likelihood:                -8559.9
No. Observations:               31365   AIC:                         1.720e+04
Df Residuals:                   31325   BIC:                         1.753e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

In [25]:
# harvest_early
print("\n--- Add harvest_early ---")
model = smf.ols(
    formula="any_burning ~ winsorized_deaths + C(year) + C(country) + harvest_early",
    data=merged_farm_df
).fit()
print(model.summary())


--- Add harvest_early ---
                            OLS Regression Results                            
Dep. Variable:            any_burning   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     196.2
Date:                Fri, 15 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:35:16   Log-Likelihood:                -7825.4
No. Observations:               28092   AIC:                         1.573e+04
Df Residuals:                   28053   BIC:                         1.605e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

In [27]:
print("\n--- Add foodshortagetime ---")
model = smf.ols(
    formula="any_burning ~ winsorized_deaths + C(year) + C(country) + foodshortagetime",
    data=merged_farm_df
).fit()
print(model.summary())


--- Add foodshortagetime ---
                            OLS Regression Results                            
Dep. Variable:            any_burning   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     179.5
Date:                Fri, 15 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:37:47   Log-Likelihood:                -8711.9
No. Observations:               31345   AIC:                         1.750e+04
Df Residuals:                   31305   BIC:                         1.784e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

In [29]:
print("\n--- Add debts_have ---")
model = smf.ols(
    formula="any_burning ~ winsorized_deaths + C(year) + C(country) + debts_have",
    data=merged_farm_df
).fit()
print(model.summary())


--- Add debts_have ---
                            OLS Regression Results                            
Dep. Variable:            any_burning   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     77.73
Date:                Fri, 15 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:38:28   Log-Likelihood:                -6461.5
No. Observations:               24900   AIC:                         1.299e+04
Df Residuals:                   24864   BIC:                         1.329e+04
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------

In [31]:
print("\n--- Add count_people ---")
model = smf.ols(
    formula="any_burning ~ winsorized_deaths + C(year) + C(country) + count_people",
    data=merged_farm_df
).fit()
print(model.summary()) 


--- Add count_people ---
                            OLS Regression Results                            
Dep. Variable:            any_burning   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     143.3
Date:                Fri, 15 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:39:00   Log-Likelihood:                -2868.4
No. Observations:               17486   AIC:                             5783.
Df Residuals:                   17463   BIC:                             5962.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------